In [1]:
!pip install unicodecsv
!pip install gensim
from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import unicodecsv                               # csv reader
import re                                       # regular expressions
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier

# To do preprocessing
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support # to report on precision and recall
import numpy as np # To compute the average results

from random import shuffle # To shuffle the dataset


# To use feature selection in the Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from nltk import sent_tokenize, word_tokenize, pos_tag
nltk.download('large_grammars')
grammar = nltk.data.load('grammars/large_grammars/atis.cfg')

  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-cp27-none-any.whl size=10771 sha256=ab66c32a4f948dfc8bd341444f630a060db08ed63d7f348c37669854e498c2b9
  Stored in directory: /root/.cache/pip/wheels/a6/09/e9/e800279c98a0a8c94543f3de6c8a562f60e51363ed26e71283
Successfully built unicodecsv
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package large_grammars to /root/nltk_data...
[nltk_data]   Unzipping grammars/large_grammars.zip.


In [0]:
# load data from the files and append it

def populate(path, reviewText=None):
    with open(path, 'rb') as f:
        reader = unicodecsv.reader(f, encoding='utf-8', delimiter=',')
        # print(reader)
        reader.next()
        for line1 in reader:
            if line1[1] not in labelMap1:
                # print(line1[1])
                labelMap1[line1[1]] = line1[1]
################## FOR GETTING GENDER FROM LINES LOAD INTO rawData (training) ########

def loadData(path, reviewText=None):
    with open(path, 'rb') as f:
        reader = unicodecsv.reader(f, encoding='utf-8', delimiter=',')
        # print(reader)
        reader.next()
        
        for line in reader:
            # if line[1] not in labelMap1:
            #   labelMap1[line[1]] = line[1]
            # print(line)
            # (reviewText, label) = parseReview(line)
            # rawData.append((reviewText, label))
            (Text, Label) = parseReviewImproved(line)
            rawData.append((Text, Label))

############## FOR GETTING GENDER FROM LINES LOAD INTO rawData1 (testing)########
def loadData1(path, reviewText=None):
    with open(path, 'rb') as f:
        reader = unicodecsv.reader(f, encoding='utf-8', delimiter=',')
        # print(reader)
        reader.next()
        for line in reader:
            # print(line)
            # (reviewText, label) = parseReview(line)
            # rawData.append((reviewText, label))
            (Text, Label) = parseReviewImproved(line)
            rawData1.append((Text, Label))

######################### FOR GETTING SPEAKER, LOADING INTO rawData2 (TRAINING) ################
def loadData2(path, reviewText=None):
    with open(path, 'rb') as f:
        reader = unicodecsv.reader(f, encoding='utf-8', delimiter=',')
        # print(reader)
        reader.next()
        for line in reader:
            # print(line)
            # (reviewText, label) = parseReview(line)
            # rawData.append((reviewText, label))
            (Text, Label) = parseReviewImproved1(line)
            rawData2.append((Text, Label))


######################### FOR GETTING SPEAKER, LOADING INTO rawData3 (Testing) ################
def loadData3(path, reviewText=None):
    with open(path, 'rb') as f:
        reader = unicodecsv.reader(f, encoding='utf-8', delimiter=',')
        # print(reader)
        reader.next()
        for line in reader:
            # print(line)
            # (reviewText, label) = parseReview(line)
            # rawData.append((reviewText, label))
            (Text, Label) = parseReviewImproved1(line)
            rawData3.append((Text, Label))




# def splitData(percentage):
#     dataSamples = len(rawData)
#     halfOfData = int(len(rawData)/2)
#     trainingSamples = int((percentage*dataSamples)/2)
#     for (Text, Label) in rawData[:trainingSamples] + rawData[halfOfData:halfOfData+trainingSamples]:
#         trainData.append((toFeatureVector(preProcess(Text)),Label))
#     for (Text, Label) in rawData[trainingSamples:halfOfData] + rawData[halfOfData+trainingSamples:]:
#         testData.append((toFeatureVector(preProcess(Text)),Label))


####### FOR GETTING GENDER FROM LINES ############################

def splitData(num):
    for (Text, Label) in rawData:
        trainData.append((toFeatureVector(preProcess(Text), num),Label))

def splitData1(num):
    for (Text, Label) in rawData1:
        testData.append((toFeatureVector(preProcess(Text), num),Label))

####### FOR GETTING SPEAKER FROM LINES ###########################
def splitData2(num):
    for (Text, Label) in rawData2:
        trainData1.append((toFeatureVector(preProcess(Text), num),Label))

def splitData3(num):
    for (Text, Label) in rawData3:
        testData1.append((toFeatureVector(preProcess(Text), num),Label))

In [0]:
# the output classes
maleLabel = 'male'
femaleLabel = 'female'
labelMap = {'male' : maleLabel, 'female' : femaleLabel}
# labelMap1 = {}

# Convert line from input file into an id/text/label tuple plus meta features
def parseReviewImproved(reviewLine):
    # Id    = int(reviewLine[0])
    # Rating = int(reviewLine[2])
    # VerifiedPurchase = reviewLine[3]
    # Category = reviewLine[4]
    Text  = reviewLine[0]
    Label = labelMap[reviewLine[2]]
    return (Text, Label)
################## FOR GETTING SPEAKER #########################
def parseReviewImproved1(reviewLine):
    # Id    = int(reviewLine[0])
    # Rating = int(reviewLine[2])
    # VerifiedPurchase = reviewLine[3]
    # Category = reviewLine[4]
    Text  = reviewLine[0]
    Label1 = labelMap1[reviewLine[1]]
    return (Text, Label1)

In [20]:
# TEXT PREPROCESSING AND FEATURE VECTORIZATION

# input: a string of one review
def preProcess(text):
    # should return a list of tokens
    
    # word tokenisation, including punctuation removal
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    tokens = pos_tag(tokens)
    
    # lowercasing
    tokens = [(t.lower(),p) for t,p in tokens]

    # stopword removal- benefits are it removes rare words, though bad for bigram relations
    if False:
        stop = set(stopwords.words('english'))
        tokens = [(t,p) for t,p in tokens if t not in stop]
    
    # lemmatisation
    lemmatiser = WordNetLemmatizer()
    tokens = [(lemmatiser.lemmatize(t),p) for t,p in tokens]
    tokens = [(t,p) for t,p in tokens if t] # ensure no empty space
    toks = [t for t,p in tokens]

    # tokens = [pos_tag(word) for word in tokens]
    # print(tokens[0], pos_tag(tokens[0]))
    # print(pos_tag(tokens))

    # tokens = pos_tag(tokens)
    
    return tokens

print(preProcess("hello this is the, ehh... presumably, a crying situations!"))

[('hello', 'NN'), ('this', 'DT'), ('is', 'VBZ'), ('the', 'DT'), ('ehh', 'NN'), ('presumably', 'RB'), ('a', 'DT'), (u'cry', 'NN'), (u'situation', 'NNS')]


In [0]:
# TEXT PREPROCESSING AND FEATURE VECTORIZATION PART 2 (WORD2VEC)
def gen_train_lib(num):
    if num == 1:
        for (Text, Label) in rawData:
            temp1 = gen_helper(Text,num)
        # genderf1 = 1

    elif num == 2:
        for (Text, Label) in rawData1:
            temp1 = gen_helper(Text, num)
        # genderf2 = 1
    
                                   
def gen_helper(text, num):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    tokens = pos_tag(tokens)
    
    # lowercasing
    tokens = [(t.lower(),p) for t,p in tokens]

    # stopword removal- benefits are it removes rare words, though bad for bigram relations
    if False:
        stop = set(stopwords.words('english'))
        tokens = [(t,p) for t,p in tokens if t not in stop]
    
    # lemmatisation
    lemmatiser = WordNetLemmatizer()
    tokens = [(lemmatiser.lemmatize(t),p) for t,p in tokens]
    tokens = [(t,p) for t,p in tokens if t] # ensure no empty space
    toks = [t for t,p in tokens]

    if num == 1:
      gender_whole_train.append(toks)
    else:
      gender_whole_test.append(toks)

def gen_train():
    gender_model_test = Word2Vec(gender_whole_test, min_count=1)
def gen_test():    
    gender_model_train = Word2Vec(gender_whole_train, min_count=1)


In [0]:
# QUESTION 2
featureDict = {} # the global feature dictionary

def toFeatureVector(tokens, num):
    # return a dictionary 'featureVect' where the keys are the tokens in 'words' and the values are the number of occurrences of the tokens
    # start by using binary values only:
#     baseDict = {}
    featureVec = {}
    prev_tag = {}

    for w, pos in tokens:
        try:
            sum = 0
            featureVec[w] += 1.0/len(tokens)
            # USING POS TAGS. COMMENT OUT THE BELOW SECTION TO OBSERVE SCORE WITHOUT POS TAGS
            if w in prev_tag:
              if pos != prev_tag[w]:
                featureVec[w] -= 1.0/len(tokens)
              else:
                featureVec[w] += 1.0/len(tokens)
            prev_tag[w] = pos
            ########################

            ## USING WORD2VEC ###############
            
            temp_word = w
            if num == 1:
                ret_list = model1.wv.most_similar(temp_word)
            elif num == 2:
                ret_list = model2.wv.most_similar(temp_word)
            length = len(ret_list)
            for x,y in ret_list:
                sum = sum + y
            avg = sum / length
            featureVec[w] += avg

            ##################################




        except KeyError:
            featureVec[w] = 1.0/len(tokens)
        try:
            featureDict[w] += 1.0/len(tokens)
        except KeyError:
            featureDict[w] = 1.0/len(tokens)
    
    # # just get bigram binary presence or not
    # for i in range(1, len(tokens)):
    #     bigram = tokens[0][i-1] + " " + tokens[0][i]
    #     try:
    #         featureVec[bigram] = 1 #+= 1.0/len(tokens)
    #     except KeyError:

    #         featureVec[bigram] = 1 #= 1.0/len(tokens)
    #     try:
    #         featureDict[bigram] += 1.0
    #     except KeyError:
    #         featureDict[bigram] = 1.0
    # print(featureVec)
    return featureVec

In [0]:
# TRAINING AND VALIDATING OUR CLASSIFIER
def trainClassifier(trainData):
    print "Training Classifier..."
    # pipeline =  Pipeline([('tfidf', TfidfTransformer()),('chi2', SelectKBest(chi2, k=948)),('svc', LinearSVC(loss = 'hinge'))])
    # pipeline =  Pipeline([('tfidf', TfidfTransformer()),('chi2', SelectKBest(chi2, k=1555)),('svc', LinearSVC(loss = 'hinge'))])
    pipeline =  Pipeline([('tfidf', TfidfTransformer()),('chi2', SelectKBest(chi2, k=1555)),('svc', LinearSVC(loss = 'squared_hinge'))])


    # print("INSIDE")
    return SklearnClassifier(pipeline).train(trainData)

In [0]:
# PREDICTING LABELS GIVEN A CLASSIFIER

def predictLabels(reviewSamples, classifier):
    return classifier.classify_many(map(lambda t: t[0], reviewSamples))

def predictLabel(text, classifier):
    return classifier.classify(toFeatureVector(preProcess(text)))

In [0]:
def crossValidate(dataset, folds):
    shuffle(dataset)
    results = []
    foldSize = len(dataset)/folds
    
    for i in range(0,len(dataset),foldSize):
        # insert code here that trains and tests on the 10 folds of data in the dataset
        print "Fold start on items %d - %d" % (i, i+foldSize)
        myTestData = dataset[i:i+foldSize]
        myTrainData = dataset[:i] + dataset[i+foldSize:]
        # print("HERE")
        classifier = trainClassifier(myTrainData)
        # print("HERE1")
        y_true = map(lambda x: x[1], myTestData)
        # print("HERE2")
        y_pred = predictLabels(myTestData, classifier)
        # print("HERE3")
        results.append(precision_recall_fscore_support(y_true, y_pred, average='weighted'))
        
    avgResults = map(np.mean,zip(*results)[:3])
    return avgResults

In [25]:
# MAIN
labelMap1 = {}
# loading reviews
rawData = [] # the filtered data from the dataset file for the training Data GETTING GENDER
trainData = [] # the training data  GETTING GENDER
testData = [] # the test data  GETTING GENDER
rawData1 = [] # the filtered data from the dataset file for the testing Data GETTING GENDER

rawData2 = [] # the filtered data from the dataset file for the training Data  GETTING SPEAKER
rawData3 = [] # the filtered data from the dataset file for the testing Data   GETTING SPEAKER
trainData1 = [] # the training data  GETTING SPEAKER
testData1 = [] # the test data  GETTING SPEAKER

gender_whole_train = []
gender_whole_test = []


# references to the data files
trainPath = '/training.csv'


testPath = '/test.csv'

# do the actual stuff
print "Now %d rawData, %d rawData1, %d trainData, %d testData" % (len(rawData), len(rawData1), len(trainData), len(testData))
print "Preparing the dataset..."

populate(trainPath)
loadData(trainPath)
loadData1(testPath)

loadData2(trainPath)
loadData3(testPath)

gen_train_lib(1)
gen_train_lib(2)

model1 = Word2Vec(gender_whole_train, min_count=1)
model2 = Word2Vec(gender_whole_test, min_count=1)

# print "Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData))
print "Now %d rawData, %d rawData1, %d trainData, %d testData" % (len(rawData), len(rawData1), len(trainData), len(testData))
print "Preparing training and test data..."
############# FOR GETTING GENDER
splitData(1)
splitData1(2)
##################### FOR GETTING SPEAKER

splitData2(1)
splitData3(2)
# print "Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData))
print "Now %d rawData, %d rawData1, %d trainData, %d testData" % (len(rawData), len(rawData1), len(trainData), len(testData))


# print "Now %d rawData2, %d trainData1, %d testData1" % (len(rawData2), len(trainData1), len(testData1))
print "Now %d rawData2, %d rawData3, %d trainData1, %d testData1" % (len(rawData2), len(rawData3), len(trainData1), len(testData1))


# print(trainData)

Now 0 rawData, 0 rawData1, 0 trainData, 0 testData
Preparing the dataset...
Now 10112 rawData, 1123 rawData1, 0 trainData, 0 testData
Preparing training and test data...


/usr/local/lib/python2.7/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Now 10112 rawData, 1123 rawData1, 10112 trainData, 1123 testData
Now 10112 rawData2, 1123 rawData3, 10112 trainData1, 1123 testData1


In [26]:
# CROSS VALIDATION FOR GENDER
print("CROSSVALIDATING FOR GENDER")
cv_results = crossValidate(trainData, 10)
print cv_results

# print("CROSSVALIDATING FOR SPEAKER")
# cv_results1 = crossValidate(trainData1, 10)
# print cv_results1

CROSSVALIDATING FOR GENDER
Fold start on items 0 - 1011
Training Classifier...
Fold start on items 1011 - 2022
Training Classifier...
Fold start on items 2022 - 3033
Training Classifier...
Fold start on items 3033 - 4044
Training Classifier...
Fold start on items 4044 - 5055
Training Classifier...
Fold start on items 5055 - 6066
Training Classifier...
Fold start on items 6066 - 7077
Training Classifier...
Fold start on items 7077 - 8088
Training Classifier...
Fold start on items 8088 - 9099
Training Classifier...
Fold start on items 9099 - 10110
Training Classifier...
Fold start on items 10110 - 11121
Training Classifier...
[0.5871007803451584, 0.5404639870515241, 0.5549874771377812]


In [27]:
####### CROSSVALIDATION FOR SPEAKER
print("CROSSVALIDATING FOR SPEAKER")
cv_results1 = crossValidate(trainData1, 10)
print cv_results1

CROSSVALIDATING FOR SPEAKER
Fold start on items 0 - 1011
Training Classifier...
Fold start on items 1011 - 2022
Training Classifier...
Fold start on items 2022 - 3033
Training Classifier...
Fold start on items 3033 - 4044
Training Classifier...
Fold start on items 4044 - 5055
Training Classifier...
Fold start on items 5055 - 6066
Training Classifier...
Fold start on items 6066 - 7077
Training Classifier...
Fold start on items 7077 - 8088
Training Classifier...
Fold start on items 8088 - 9099
Training Classifier...
Fold start on items 9099 - 10110
Training Classifier...
Fold start on items 10110 - 11121
Training Classifier...
[0.18154126598572554, 0.1766927434583221, 0.1564397853007664]


In [28]:
##### PREDICTION FOR GENDER
print("PREDICITING GENDER")
classifier = trainClassifier(trainData)
testTrue = map(lambda t: t[1], testData)
testPred = predictLabels(testData, classifier)
finalScores = precision_recall_fscore_support(testTrue, testPred, average='weighted')
finalScoresraw = precision_recall_fscore_support(testTrue, testPred)
print "Done training!"
print("Raw results for Gender = ", finalScoresraw)
print "Precision: %f\nRecall: %f\nF Score:%f" % finalScores[:3]
# print(labelMap)
# print(labelMap1)

PREDICITING GENDER
Training Classifier...
Done training!
('Raw results for Gender = ', (array([0.5625    , 0.61211129]), array([0.54857143, 0.62541806]), array([0.55544841, 0.61869313]), array([525, 598])))
Precision: 0.588918
Recall: 0.589492
F Score:0.589126


In [29]:
##### PREDICTION FOR SPEAKER
print("PREDICTING SPEAKER")
classifier1 = trainClassifier(trainData1)
testTrue1 = map(lambda t: t[1], testData1)
testPred1 = predictLabels(testData1, classifier1)
finalScores1 = precision_recall_fscore_support(testTrue1, testPred1, average='weighted')
finalScoresraw1 = precision_recall_fscore_support(testTrue1, testPred1)

print "Done training!"
# print("Raw results for speaker = ", finalScoresraw1)
print "Precision1: %f\nRecall1: %f\nF Score1:%f" % finalScores1[:3]
# print(labelMap)
# print(labelMap1)

PREDICTING SPEAKER
Training Classifier...
Done training!
Precision1: 0.231115
Recall1: 0.204809
F Score1:0.186925
